## 1. Cargar y Explorar Dataset EPL

In [2]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurar estilos
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print('✅ Librerías importadas correctamente')

✅ Librerías importadas correctamente


In [3]:
# Cargar el dataset EPL
data_path = Path('../data/raw/epl_final.csv')

if data_path.exists():
    df = pd.read_csv(data_path)
    print(f'✅ Dataset cargado: {df.shape[0]} filas × {df.shape[1]} columnas')
    print(f'\n📅 Rango temporal: {df.iloc[:, 0] if "Date" in str(df.columns) else "Verificar columnas"}')
else:
    print(f'❌ Archivo no encontrado en {data_path}')
    print(f'   Descarga epl_final.csv desde Kaggle y colócalo en: data/raw/')
    df = None

✅ Dataset cargado: 9380 filas × 22 columnas

📅 Rango temporal: 0       2000/01
1       2000/01
2       2000/01
3       2000/01
4       2000/01
         ...   
9375    2024/25
9376    2024/25
9377    2024/25
9378    2024/25
9379    2024/25
Name: Season, Length: 9380, dtype: object


In [5]:
# Inspeccionar estructura del dataset
if df is not None:
    print('\n' + '='*70)
    print('ESTRUCTURA DEL DATASET')
    print('='*70)
    
    print(f'\n📋 Columnas ({len(df.columns)}):')  
    for i, col in enumerate(df.columns, 1):
        dtype = df[col].dtype
        nulls = df[col].isnull().sum()
        print(f'   {i:2d}. {col:<25} {str(dtype):<15} (nulls: {nulls:4d})')
    
    print(f'\n📊 Primeras 5 filas:')
    display(df.head())
    
    print(f'\n📊 Info del Dataset:')
    print(df.info())


ESTRUCTURA DEL DATASET

📋 Columnas (22):
    1. Season                    object          (nulls:    0)
    2. MatchDate                 object          (nulls:    0)
    3. HomeTeam                  object          (nulls:    0)
    4. AwayTeam                  object          (nulls:    0)
    5. FullTimeHomeGoals         int64           (nulls:    0)
    6. FullTimeAwayGoals         int64           (nulls:    0)
    7. FullTimeResult            object          (nulls:    0)
    8. HalfTimeHomeGoals         int64           (nulls:    0)
    9. HalfTimeAwayGoals         int64           (nulls:    0)
   10. HalfTimeResult            object          (nulls:    0)
   11. HomeShots                 int64           (nulls:    0)
   12. AwayShots                 int64           (nulls:    0)
   13. HomeShotsOnTarget         int64           (nulls:    0)
   14. AwayShotsOnTarget         int64           (nulls:    0)
   15. HomeCorners               int64           (nulls:    0)
   16. AwayCo

,Season,MatchDate,HomeTeam,AwayTeam,FullTimeHomeGoals,FullTimeAwayGoals,FullTimeResult,HalfTimeHomeGoals,HalfTimeAwayGoals,HalfTimeResult,...,HomeShotsOnTarget,AwayShotsOnTarget,HomeCorners,AwayCorners,HomeFouls,AwayFouls,HomeYellowCards,AwayYellowCards,HomeRedCards,AwayRedCards
0,2000/01,2000-08-19,Charlton,Man City,4,0,H,2,0,H,...,14,4,6,6,13,12,1,2,0,0
1,2000/01,2000-08-19,Chelsea,West Ham,4,2,H,1,0,H,...,10,5,7,7,19,14,1,2,0,0
2,2000/01,2000-08-19,Coventry,Middlesbrough,1,3,A,1,1,D,...,3,9,8,4,15,21,5,3,1,0
3,2000/01,2000-08-19,Derby,Southampton,2,2,D,1,2,A,...,4,6,5,8,11,13,1,1,0,0
4,2000/01,2000-08-19,Leeds,Everton,2,0,H,2,0,H,...,8,6,6,4,21,20,1,3,0,0



📊 Info del Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9380 entries, 0 to 9379
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Season             9380 non-null   object
 1   MatchDate          9380 non-null   object
 2   HomeTeam           9380 non-null   object
 3   AwayTeam           9380 non-null   object
 4   FullTimeHomeGoals  9380 non-null   int64 
 5   FullTimeAwayGoals  9380 non-null   int64 
 6   FullTimeResult     9380 non-null   object
 7   HalfTimeHomeGoals  9380 non-null   int64 
 8   HalfTimeAwayGoals  9380 non-null   int64 
 9   HalfTimeResult     9380 non-null   object
 10  HomeShots          9380 non-null   int64 
 11  AwayShots          9380 non-null   int64 
 12  HomeShotsOnTarget  9380 non-null   int64 
 13  AwayShotsOnTarget  9380 non-null   int64 
 14  HomeCorners        9380 non-null   int64 
 15  AwayCorners        9380 non-null   int64 
 16  HomeFouls          93

## 2. Preprocesamiento y Feature Engineering

In [6]:
if df is not None:
    # Hacer una copia para trabajar
    df_processed = df.copy()
    
    # Identificar columnas de fecha
    date_cols = [col for col in df_processed.columns if 'date' in col.lower()]
    print(f'Columnas de fecha encontradas: {date_cols}')
    
    # Convertir a datetime si es necesario
    for col in date_cols:
        df_processed[col] = pd.to_datetime(df_processed[col], errors='coerce')
    
    # Verificar valores nulos
    print(f'\n📊 Valores nulos por columna:')
    null_counts = df_processed.isnull().sum()
    null_counts = null_counts[null_counts > 0].sort_values(ascending=False)
    if len(null_counts) > 0:
        print(null_counts)
    else:
        print('✅ Sin valores nulos')
    
    print(f'\n✅ Preprocesamiento inicial completado')

Columnas de fecha encontradas: ['MatchDate']

📊 Valores nulos por columna:
✅ Sin valores nulos

✅ Preprocesamiento inicial completado


In [7]:
# Feature Engineering - Crear features derivadas
if df_processed is not None:
    # Extraer características de tiempo
    date_col = date_cols[0] if date_cols else None
    
    if date_col:
        df_processed['Year'] = df_processed[date_col].dt.year
        df_processed['Month'] = df_processed[date_col].dt.month
        df_processed['DayOfWeek'] = df_processed[date_col].dt.dayofweek
        df_processed['Season'] = df_processed['Year'].apply(lambda x: f'{x}-{x+1}')
        print(f'✅ Features temporales creadas')
    
    # Identificar columnas de equipo y resultado
    print(f'\nColumnas disponibles:')
    print(df_processed.columns.tolist())

✅ Features temporales creadas

Columnas disponibles:
['Season', 'MatchDate', 'HomeTeam', 'AwayTeam', 'FullTimeHomeGoals', 'FullTimeAwayGoals', 'FullTimeResult', 'HalfTimeHomeGoals', 'HalfTimeAwayGoals', 'HalfTimeResult', 'HomeShots', 'AwayShots', 'HomeShotsOnTarget', 'AwayShotsOnTarget', 'HomeCorners', 'AwayCorners', 'HomeFouls', 'AwayFouls', 'HomeYellowCards', 'AwayYellowCards', 'HomeRedCards', 'AwayRedCards', 'Year', 'Month', 'DayOfWeek']


## 3. Análisis Exploratorio (EDA)

In [8]:
if df_processed is not None:
    print('\n' + '='*70)
    print('ANÁLISIS EXPLORATORIO')
    print('='*70)
    
    # Estadísticas básicas
    print('\n📊 Estadísticas descriptivas:')
    display(df_processed.describe())


ANÁLISIS EXPLORATORIO

📊 Estadísticas descriptivas:


,MatchDate,FullTimeHomeGoals,FullTimeAwayGoals,HalfTimeHomeGoals,HalfTimeAwayGoals,HomeShots,AwayShots,HomeShotsOnTarget,AwayShotsOnTarget,HomeCorners,AwayCorners,HomeFouls,AwayFouls,HomeYellowCards,AwayYellowCards,HomeRedCards,AwayRedCards,Year,Month,DayOfWeek
count,9380,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000,9380.000000
mean,2013-01-17 13:05:33.134328576,1.535394,1.182623,0.688273,0.518763,13.617484,10.810661,5.973134,4.693710,6.040299,4.774733,11.276333,11.765672,1.468124,1.792431,0.062473,0.085288,2012.529104,6.733902,4.448507
min,2000-08-19 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2000.000000,1.000000,0.000000
25%,2006-12-05 18:00:00,1.000000,0.000000,0.000000,0.000000,10.000000,7.000000,4.000000,3.000000,4.000000,3.000000,9.000000,9.000000,1.000000,1.000000,0.000000,0.000000,2006.000000,3.000000,5.000000
50%,2013-01-19 00:00:00,1.000000,1.000000,0.000000,0.000000,13.000000,10.000000,6.000000,4.000000,6.000000,4.000000,11.000000,12.000000,1.000000,2.000000,0.000000,0.000000,2013.000000,8.000000,5.000000
75%,2019-03-02 00:00:00,2.000000,2.000000,1.000000,1.000000,17.000000,14.000000,8.000000,6.000000,8.000000,6.000000,14.000000,14.000000,2.000000,3.000000,0.000000,0.000000,2019.000000,10.000000,5.000000
max,2025-05-05 00:00:00,9.000000,9.000000,5.000000,5.000000,43.000000,37.000000,24.000000,20.000000,20.000000,19.000000,33.000000,29.000000,7.000000,9.000000,3.000000,2.000000,2025.000000,12.000000,6.000000
std,NaN,1.305432,1.157414,0.835079,0.735357,5.356424,4.696501,3.267954,2.750045,3.110619,2.749541,3.750250,3.924179,1.217190,1.288481,0.253221,0.290183,7.213963,3.928129,1.731962


## 2.1 Análisis de Target Variables

In [9]:
# Analizar los targets
if df_processed is not None:
    print('📊 DISTRIBUCIÓN DE RESULTADOS (1X2):')
    result_counts = df_processed['FullTimeResult'].value_counts()
    result_pct = df_processed['FullTimeResult'].value_counts(normalize=True) * 100
    
    for result in ['H', 'D', 'A']:
        if result in result_counts.index:
            count = result_counts[result]
            pct = result_pct[result]
            label = 'Home Win' if result == 'H' else 'Draw' if result == 'D' else 'Away Win'
            print(f"  {label:12} ('{result}'): {count:5d} ({pct:5.1f}%)")
    
    print('\n📊 DISTRIBUCIÓN DE GOLES TOTALES:')
    df_processed['TotalGoals'] = df_processed['FullTimeHomeGoals'] + df_processed['FullTimeAwayGoals']
    
    print(f"  Mínimo: {df_processed['TotalGoals'].min()}")
    print(f"  Máximo: {df_processed['TotalGoals'].max()}")
    print(f"  Promedio: {df_processed['TotalGoals'].mean():.2f}")
    print(f"  Mediana: {df_processed['TotalGoals'].median():.1f}")
    
    print('\n  Distribución:')
    print(df_processed['TotalGoals'].value_counts().sort_index())

📊 DISTRIBUCIÓN DE RESULTADOS (1X2):
  Home Win     ('H'):  4299 ( 45.8%)
  Draw         ('D'):  2313 ( 24.7%)
  Away Win     ('A'):  2768 ( 29.5%)

📊 DISTRIBUCIÓN DE GOLES TOTALES:
  Mínimo: 0
  Máximo: 11
  Promedio: 2.72
  Mediana: 3.0

  Distribución:
TotalGoals
0      692
1     1626
2     2238
3     2057
4     1476
5      758
6      312
7      143
8       54
9       18
10       5
11       1
Name: count, dtype: int64


In [10]:
# Visualizar distribuciones
if df_processed is not None:
    # Encontrar columnas numéricas potencialmente interesantes
    numeric_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
    
    print(f'Columnas numéricas: {numeric_cols[:10]}...')
    print(f'\nTotal de columnas numéricas: {len(numeric_cols)}')

Columnas numéricas: ['FullTimeHomeGoals', 'FullTimeAwayGoals', 'HalfTimeHomeGoals', 'HalfTimeAwayGoals', 'HomeShots', 'AwayShots', 'HomeShotsOnTarget', 'AwayShotsOnTarget', 'HomeCorners', 'AwayCorners']...

Total de columnas numéricas: 20


## 3. Feature Engineering - Crear Variables Predictivas

### Qué son Features?
Son variables que usa el modelo ML para predecir. Pueden ser:
- **Features base**: Las que ya existen en el dataset (Shots, Fouls, etc)
- **Features derivadas**: Que creamos combinando info histórica (Form, H2H, etc)

### Features que Vamos a Crear:

1. **Form**: Puntos de últimos 5 partidos (3=win, 1=draw, 0=loss)
2. **Head-to-Head (H2H)**: % victorias históricas entre equipos
3. **Goles Promedio**: Media de goles a favor/contra
4. **Home Advantage**: Ventaja de jugar en casa
5. **Temporales**: Mes, día semana, año

In [11]:
import sys
sys.path.insert(0, '../src')

try:
    from src.feature_engineering import EPLFeatureEngineer, prepare_training_data
except ModuleNotFoundError:
    # Fallback: cargar directamente desde el archivo ../src/feature_engineering.py
    import importlib.util
    from pathlib import Path

    fe_path = Path('../src/feature_engineering.py').resolve()
    if fe_path.exists():
        spec = importlib.util.spec_from_file_location('feature_engineering', str(fe_path))
        fe_mod = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(fe_mod)
        EPLFeatureEngineer = getattr(fe_mod, 'EPLFeatureEngineer')
        prepare_training_data = getattr(fe_mod, 'prepare_training_data')
    else:
        # Si no existe el archivo, relanzamos el error original para que el usuario lo vea
        raise

if df_processed is not None:
    print('🔧 CREANDO FEATURES INGENIERILES...\n')
    
    # Crear el ingeniero de features
    engineer = EPLFeatureEngineer(df_processed)
    
    # Crear todas las features
    X, y_result, y_goals = engineer.engineer_features()
    
    print(f'\n✅ Features creadas exitosamente!')
    print(f'   Dimensiones X: {X.shape}')
    print(f'   - {X.shape[0]} muestras (partidos)')
    print(f'   - {X.shape[1]} features (variables)')
    
    print(f'\n📊 Target Variables:')
    print(f'   - y_result: Resultado (0=Away, 1=Draw, 2=Home)')
    print(f'   - y_goals: Goles totales')

🔧 CREANDO FEATURES INGENIERILES...

🔧 Creando features...
  → Form de equipos...
  → Estadísticas de goles...
  → Ventaja de casa...
  → Estadísticas de goles...
  → Ventaja de casa...
  → Estadísticas de tiros...
✅ Features creadas: 208 columnas

✅ Features creadas exitosamente!
   Dimensiones X: (9380, 208)
   - 9380 muestras (partidos)
   - 208 features (variables)

📊 Target Variables:
   - y_result: Resultado (0=Away, 1=Draw, 2=Home)
   - y_goals: Goles totales
  → Estadísticas de tiros...
✅ Features creadas: 208 columnas

✅ Features creadas exitosamente!
   Dimensiones X: (9380, 208)
   - 9380 muestras (partidos)
   - 208 features (variables)

📊 Target Variables:
   - y_result: Resultado (0=Away, 1=Draw, 2=Home)
   - y_goals: Goles totales


In [12]:
# Inspeccionar las features creadas
if X is not None:
    print('📋 COLUMNAS DE FEATURES CREADAS:')
    print(f'\nTotal: {len(X.columns)} features\n')
    
    for i, col in enumerate(X.columns, 1):
        dtype = X[col].dtype
        nulls = X[col].isnull().sum()
        print(f'   {i:2d}. {col:<30} {str(dtype):<10} (nulls: {nulls:4d})')
    
    print(f'\n📊 ESTADÍSTICAS DE FEATURES:')
    print(X.describe().T[['count', 'mean', 'std', 'min', 'max']].head(10))

📋 COLUMNAS DE FEATURES CREADAS:

Total: 208 features

    1. HomeShots                      int64      (nulls:    0)
    2. AwayShots                      int64      (nulls:    0)
    3. HomeShotsOnTarget              int64      (nulls:    0)
    4. AwayShotsOnTarget              int64      (nulls:    0)
    5. HomeCorners                    int64      (nulls:    0)
    6. AwayCorners                    int64      (nulls:    0)
    7. HomeFouls                      int64      (nulls:    0)
    8. AwayFouls                      int64      (nulls:    0)
    9. HomeYellowCards                int64      (nulls:    0)
   10. AwayYellowCards                int64      (nulls:    0)
   11. HomeRedCards                   int64      (nulls:    0)
   12. AwayRedCards                   int64      (nulls:    0)
   13. HalfTimeHomeGoals              int64      (nulls:    0)
   14. HalfTimeAwayGoals              int64      (nulls:    0)
   15. HomeTeam_Form                  float64    (nulls:    0)
 

## 4. Preparar Datos para Modelado

## 4. Preparar Datos para Modelado

In [13]:
# Preparar datos para entrenamiento
if X is not None and y_result is not None:
    from sklearn.preprocessing import StandardScaler
    
    print('📊 PREPARANDO DATOS...\n')
    
    # Llenar NaNs con forward fill
    X_filled = X.fillna(method='ffill').fillna(method='bfill')
    
    print(f'✅ NaNs llenos:')
    print(f'   Nulls restantes en X: {X_filled.isnull().sum().sum()}')
    
    # Split temporal (no aleatorio para series de tiempo)
    # Usamos 80/20 con orden temporal
    split_idx = int(len(X_filled) * 0.8)
    
    X_train = X_filled[:split_idx]
    X_test = X_filled[split_idx:]
    
    y_result_train = y_result[:split_idx]
    y_result_test = y_result[split_idx:]
    
    y_goals_train = y_goals[:split_idx]
    y_goals_test = y_goals[split_idx:]
    
    print(f'\n✅ Split Train/Test (temporal):')
    print(f'   Train: {len(X_train):,} partidos')
    print(f'   Test:  {len(X_test):,} partidos')
    
    # Normalizar features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f'\n✅ Features normalizadas (media=0, std=1)')
    
    # Guardar para las siguientes fases
    print(f'\n📦 Datos listos para modelos ML')

📊 PREPARANDO DATOS...

✅ NaNs llenos:
   Nulls restantes en X: 0

✅ Split Train/Test (temporal):
   Train: 7,504 partidos
   Test:  1,876 partidos

✅ Features normalizadas (media=0, std=1)

📦 Datos listos para modelos ML


In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

if df_processed is not None:
    print('Datos listos para la siguiente fase:')
    print(f'  - {df_processed.shape[0]} muestras')
    print(f'  - {df_processed.shape[1]} features')
    print(f'\n✅ Próximas fases: Análisis de target variables')

Datos listos para la siguiente fase:
  - 9380 muestras
  - 26 features

✅ Próximas fases: Análisis de target variables


## 5. Modelo de Predicción de Resultados (1X2)

In [16]:
# Entrenar modelos de resultado
if X_train_scaled is not None and y_result_train is not None:
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
    
    print('🤖 ENTRENANDO MODELOS - PREDICCIÓN DE RESULTADOS (1X2)\n')
    
    # Random Forest
    print('1️⃣  Random Forest Classifier')
    rf_result = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
    rf_result.fit(X_train_scaled, y_result_train)
    y_pred_rf = rf_result.predict(X_test_scaled)
    
    rf_acc = accuracy_score(y_result_test, y_pred_rf)
    rf_f1 = f1_score(y_result_test, y_pred_rf, average='weighted')
    print(f'   Accuracy: {rf_acc:.4f} ({rf_acc*100:.2f}%)')
    print(f'   F1-Score: {rf_f1:.4f}')
    
    # Gradient Boosting
    print('\n2️⃣  Gradient Boosting Classifier')
    gb_result = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
    gb_result.fit(X_train_scaled, y_result_train)
    y_pred_gb = gb_result.predict(X_test_scaled)
    
    gb_acc = accuracy_score(y_result_test, y_pred_gb)
    gb_f1 = f1_score(y_result_test, y_pred_gb, average='weighted')
    print(f'   Accuracy: {gb_acc:.4f} ({gb_acc*100:.2f}%)')
    print(f'   F1-Score: {gb_f1:.4f}')
    
    # Comparación
    print('\n' + '-'*60)
    if gb_acc > rf_acc:
        print(f'🏆 Mejor: Gradient Boosting ({gb_acc:.4f})')
        mejor_resultado = 'GB'
    else:
        print(f'🏆 Mejor: Random Forest ({rf_acc:.4f})')
        mejor_resultado = 'RF'
    print('-'*60)

🤖 ENTRENANDO MODELOS - PREDICCIÓN DE RESULTADOS (1X2)

1️⃣  Random Forest Classifier
   Accuracy: 0.6274 (62.74%)
   F1-Score: 0.5805

2️⃣  Gradient Boosting Classifier
   Accuracy: 0.6274 (62.74%)
   F1-Score: 0.5805

2️⃣  Gradient Boosting Classifier
   Accuracy: 0.5800 (58.00%)
   F1-Score: 0.5983

------------------------------------------------------------
🏆 Mejor: Random Forest (0.6274)
------------------------------------------------------------
   Accuracy: 0.5800 (58.00%)
   F1-Score: 0.5983

------------------------------------------------------------
🏆 Mejor: Random Forest (0.6274)
------------------------------------------------------------


In [17]:
# Entrenar modelos de goles totales
if X_train_scaled is not None and y_goals_train is not None:
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    import numpy as np
    
    print('🤖 ENTRENANDO MODELOS - PREDICCIÓN DE GOLES TOTALES\n')
    
    # Random Forest Regressor
    print('1️⃣  Random Forest Regressor')
    rf_goals = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
    rf_goals.fit(X_train_scaled, y_goals_train)
    y_pred_rf_goals = rf_goals.predict(X_test_scaled)
    
    rf_mae = mean_absolute_error(y_goals_test, y_pred_rf_goals)
    rf_rmse = np.sqrt(mean_squared_error(y_goals_test, y_pred_rf_goals))
    rf_r2 = r2_score(y_goals_test, y_pred_rf_goals)
    print(f'   MAE: {rf_mae:.4f}')
    print(f'   RMSE: {rf_rmse:.4f}')
    print(f'   R²: {rf_r2:.4f}')
    
    # Gradient Boosting Regressor
    print('\n2️⃣  Gradient Boosting Regressor')
    gb_goals = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
    gb_goals.fit(X_train_scaled, y_goals_train)
    y_pred_gb_goals = gb_goals.predict(X_test_scaled)
    
    gb_mae = mean_absolute_error(y_goals_test, y_pred_gb_goals)
    gb_rmse = np.sqrt(mean_squared_error(y_goals_test, y_pred_gb_goals))
    gb_r2 = r2_score(y_goals_test, y_pred_gb_goals)
    print(f'   MAE: {gb_mae:.4f}')
    print(f'   RMSE: {gb_rmse:.4f}')
    print(f'   R²: {gb_r2:.4f}')
    
    # Comparación
    print('\n' + '-'*60)
    if gb_r2 > rf_r2:
        print(f'🏆 Mejor: Gradient Boosting (R²: {gb_r2:.4f})')
        mejor_goles = 'GB'
    else:
        print(f'🏆 Mejor: Random Forest (R²: {rf_r2:.4f})')
        mejor_goles = 'RF'
    print('-'*60)

🤖 ENTRENANDO MODELOS - PREDICCIÓN DE GOLES TOTALES

1️⃣  Random Forest Regressor
   MAE: 0.9654
   RMSE: 1.1882
   R²: 0.5125

2️⃣  Gradient Boosting Regressor
   MAE: 0.9584
   RMSE: 1.1843
   R²: 0.5157

------------------------------------------------------------
🏆 Mejor: Gradient Boosting (R²: 0.5157)
------------------------------------------------------------


## Sección 6: Guardar Modelos para Uso Futuro

In [26]:
# Guardar modelos para usar después
import pickle
import os
from pathlib import Path

# Obtener directorio actual y crear ruta absoluta
current_dir = Path.cwd()
models_dir = current_dir / 'models'

print(f'📂 Directorio actual: {current_dir}')
print(f'📂 Guardando en: {models_dir}')

# Crear carpeta models/ si no existe
models_dir.mkdir(exist_ok=True)

# Guardar los 4 modelos + scaler
models_to_save = {
    'rf_result': rf_result,           # Random Forest - Predicción 1X2
    'gb_result': gb_result,           # Gradient Boosting - Predicción 1X2
    'rf_goals': rf_goals,             # Random Forest - Predicción Goles
    'gb_goals': gb_goals,             # Gradient Boosting - Predicción Goles
    'scaler': scaler                  # StandardScaler - Normalización de features
}

print('\n💾 GUARDANDO MODELOS ENTRENADOS...\n')
for name, model in models_to_save.items():
    filepath = models_dir / f'{name}_model.pkl'
    with open(filepath, 'wb') as f:
        pickle.dump(model, f)
    print(f'✅ {filepath}')

print('\n🎉 Todos los modelos guardados correctamente')
print(f'📂 Ubicación: {models_dir}/')

# Verificar que existen
print(f'\n✓ Verificación:')
for file in models_dir.glob('*.pkl'):
    size = file.stat().st_size / (1024*1024)  # Size en MB
    print(f'  {file.name}: {size:.1f} MB')

📂 Directorio actual: /workspaces/codespaces-blank/premier-league-ml/notebooks
📂 Guardando en: /workspaces/codespaces-blank/premier-league-ml/notebooks/models

💾 GUARDANDO MODELOS ENTRENADOS...

✅ /workspaces/codespaces-blank/premier-league-ml/notebooks/models/rf_result_model.pkl
✅ /workspaces/codespaces-blank/premier-league-ml/notebooks/models/gb_result_model.pkl
✅ /workspaces/codespaces-blank/premier-league-ml/notebooks/models/rf_goals_model.pkl
✅ /workspaces/codespaces-blank/premier-league-ml/notebooks/models/gb_goals_model.pkl
✅ /workspaces/codespaces-blank/premier-league-ml/notebooks/models/scaler_model.pkl

🎉 Todos los modelos guardados correctamente
📂 Ubicación: /workspaces/codespaces-blank/premier-league-ml/notebooks/models/

✓ Verificación:
  gb_goals_model.pkl: 0.4 MB
  gb_result_model.pkl: 1.2 MB
  scaler_model.pkl: 0.0 MB
  rf_result_model.pkl: 17.7 MB
  rf_goals_model.pkl: 16.1 MB


## Sección 7: Hacer Predicciones en Nuevos Partidos

In [24]:
# Cargar el módulo de predicción
import sys
from pathlib import Path
import importlib

# Agregar src al path para importar módulos
sys.path.insert(0, str(Path.cwd() / 'src'))

# Recargar módulos para asegurar cambios recientes
import predictor
importlib.reload(predictor)
from predictor import EPLPredictor

# Inicializar predictor (carga modelos desde ./models/)
predictor_instance = EPLPredictor('models')

# Hacer predicción para un partido futuro
print('='*70)
print('🔮 EJEMPLO DE PREDICCIÓN - Nuevo Partido')
print('='*70)

# Predecir: Chelsea vs Liverpool el 22 de Febrero, 2025
# Pasamos X_train_scaled para que el predictor use la distribución media
prediccion = predictor_instance.predict_match(
    df_historical=df,
    home_team='Chelsea',
    away_team='Liverpool',
    match_date='2025-02-22',
    X_train_scaled=X_train_scaled
)

# Mostrar resultados
predictor_instance.print_prediction(prediccion, verbose=True)

# Acceder a valores específicos
print('💡 Acceso programático a resultados:')
print(f'\nPredicción RF (1X2): {prediccion["resultado"]["random_forest"]["prediccion"]}')
print(f'Confianza: {prediccion["resultado"]["random_forest"]["confianza"]:.1f}%')
print(f'Goles totales (promedio): {prediccion["goles_totales"]["promedio"]}')

✅ Modelos cargados desde: models
🔮 EJEMPLO DE PREDICCIÓN - Nuevo Partido

🔮 PREDICCIÓN EPL
📅 Chelsea vs Liverpool (2025-02-22)

📊 RESULTADO (1X2):

  🌲 Random Forest:
     Predicción: Home Win
     Confianza: 71.3%
     Detalles: Away 14.4% | Draw 14.3% | Home 71.3%

  ⚡ Gradient Boosting:
     Predicción: Home Win
     Confianza: 73.9%
     Detalles: Away 6.8% | Draw 19.3% | Home 73.9%

⚽ GOLES TOTALES:
  🌲 Random Forest: 2.24
  ⚡ Gradient Boosting: 2.41
  📈 Promedio: 2.33


💡 Acceso programático a resultados:

Predicción RF (1X2): Home Win
Confianza: 71.3%
Goles totales (promedio): 2.33


In [28]:
# Predicciones múltiples (Batch)
import importlib
importlib.reload(predictor)
from predictor import EPLPredictor

# Recargar predictor
predictor_instance = EPLPredictor('models')

print('='*70)
print('🔮 EJEMPLO: Predicciones Múltiples (Fin de Semana)')
print('='*70)

matches = [
    {'home': 'Chelsea', 'away': 'Liverpool', 'date': '2025-02-22'},
    {'home': 'Arsenal', 'away': 'Man City', 'date': '2025-03-01'},
    {'home': 'Tottenham', 'away': 'Man United', 'date': '2025-03-08'},
]

print('\n🤖 Realizando predicciones...\n')
predictions = predictor_instance.predict_batch(df, matches, X_train_scaled)

# Mostrar resumen
print('\n' + '='*70)
print('📊 RESUMEN DE PREDICCIONES')
print('='*70 + '\n')

for pred in predictions:
    print(f"{pred['partido']} ({pred['fecha']})")
    print(f"  🌲 RF: {pred['resultado']['random_forest']['prediccion']} ({pred['resultado']['random_forest']['confianza']:.1f}%)")
    print(f"  ⚡ GB: {pred['resultado']['gradient_boosting']['prediccion']} ({pred['resultado']['gradient_boosting']['confianza']:.1f}%)")
    print(f"  ⚽ Goles: {pred['goles_totales']['promedio']}")
    print()


✅ Modelos cargados desde: models
🔮 EJEMPLO: Predicciones Múltiples (Fin de Semana)

🤖 Realizando predicciones...

✅ Chelsea vs Liverpool
✅ Arsenal vs Man City
✅ Tottenham vs Man United

📊 RESUMEN DE PREDICCIONES

Chelsea vs Liverpool (2025-02-22)
  🌲 RF: Home Win (71.3%)
  ⚡ GB: Home Win (73.9%)
  ⚽ Goles: 2.33

Arsenal vs Man City (2025-03-01)
  🌲 RF: Home Win (71.3%)
  ⚡ GB: Home Win (73.9%)
  ⚽ Goles: 2.33

Tottenham vs Man United (2025-03-08)
  🌲 RF: Home Win (71.3%)
  ⚡ GB: Home Win (73.9%)
  ⚽ Goles: 2.33



## 6. Modelo de Predicción de Goles Totales

In [11]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print('📊 Modelos de predicción de goles preparados')
print('  - Random Forest Regressor')
print('  - Gradient Boosting Regressor')

📊 Modelos de predicción de goles preparados
  - Random Forest Regressor
  - Gradient Boosting Regressor


## 7. Evaluación y Comparación de Modelos

In [12]:
print('✅ Fase de evaluación lista')
print('Métricas a evaluar:')
print('  Clasificación: Accuracy, Precision, Recall, F1-Score, ROC-AUC')
print('  Regresión: MAE, RMSE, R²')

✅ Fase de evaluación lista
Métricas a evaluar:
  Clasificación: Accuracy, Precision, Recall, F1-Score, ROC-AUC
  Regresión: MAE, RMSE, R²


## 8. Comparar Predicciones vs Odds del Mercado

In [13]:
print('📊 Investigar APIs de odds disponibles:')
print('  1. odds-api.com - Gratis con límite de requests')
print('  2. RapidAPI - Múltiples endpoints de apuestas')
print('  3. Datos históricos de sitios especializados')

📊 Investigar APIs de odds disponibles:
  1. odds-api.com - Gratis con límite de requests
  2. RapidAPI - Múltiples endpoints de apuestas
  3. Datos históricos de sitios especializados


## 9. Identificar Oportunidades de Value Betting

In [14]:
print('💡 Estrategia de Value Betting:')
print('\n1. Calcular probabilidad implícita de odds:')
print('   Prob_implícita = 1 / Odd')
print('\n2. Comparar con probabilidad predicha por modelo:')
print('   Si Prob_modelo > Prob_implícita → Posible value bet')
print('\n3. Filtrar por edge mínimo (ej: 5%)')
print('   Edge = Prob_modelo - Prob_implícita')

💡 Estrategia de Value Betting:

1. Calcular probabilidad implícita de odds:
   Prob_implícita = 1 / Odd

2. Comparar con probabilidad predicha por modelo:
   Si Prob_modelo > Prob_implícita → Posible value bet

3. Filtrar por edge mínimo (ej: 5%)
   Edge = Prob_modelo - Prob_implícita


## 10. Análisis de Rentabilidad

In [15]:
print('💰 Métricas de Rentabilidad:')
print('  - ROI (Return on Investment)')
print('  - Win Rate (%)')
print('  - Valor Esperado (EV) por apuesta')
print('  - Backtesting en datos históricos')
print('  - Análisis de rentabilidad por temporada')

💰 Métricas de Rentabilidad:
  - ROI (Return on Investment)
  - Win Rate (%)
  - Valor Esperado (EV) por apuesta
  - Backtesting en datos históricos
  - Análisis de rentabilidad por temporada
